In [1]:
import pandas as pd
from pathlib import Path
import PC_Fns
import numpy as np

In [17]:
wkdir = Path(r"C:\Users\ucasbwh\Desktop\20190923_101047_WAC_L10_20C_ABSR")

files_tc = PC_Fns.Find_Files(wkdir, "RMAP_CMD_*.txt")
print(len(files_tc))

1


In [20]:
tc_hdr_strings = ['Date', 'Time', 'Description']
tc_hdr_cmd = list(range(0,11))
tc_hdr = tc_hdr_strings + tc_hdr_cmd
tc_col = tc_hdr_cmd
tc = pd.DataFrame()

In [21]:
for curfile in files_tc:
    dt = pd.read_csv(curfile, sep='\t', names=tc_hdr, dtype=object)
    if dt.size == 0: continue
    if dt['Description'].iloc[-1] == '  ': dt = dt[:-1]
    dt[tc_col] = dt[tc_col].fillna('-1')
    dt[tc_col] = dt[tc_col].applymap(lambda x: int(x, base=16))
    dt[tc_col] = dt[tc_col].replace('-1', np.nan)
    dt['DT'] = dt['Date'] + dt['Time']
    dt['BID'] = ((dt[0]*256+dt[1])&0x7F8).values>>3
    dt['DT'] = pd.to_datetime(dt['DT'], format='%Y-%m-%d%H:%M:%S.%f ')
    tc = tc.append(dt, ignore_index = True)

tc['ACTION'] = tc['Description'].map(lambda x: x[12:])

tc.head(10)

,Date,Time,Description,0,1,2,3,4,5,6,7,8,9,10,DT,BID,ACTION
0,2019-09-23,10:11:04.334,PAN_TC_PIU_STIME,168,0,1,0,6,0,30,225,184,0,85,2019-09-23 10:11:04.334,0,STIME
1,2019-09-23,10:11:59.671,PAN_TC_PIU_CE,168,8,2,0,5,1,0,0,0,0,-1,2019-09-23 10:11:59.671,1,CE
2,2019-09-23,10:12:01.671,PAN_TC_PIU_PE,168,16,3,0,5,1,0,0,0,0,-1,2019-09-23 10:12:01.671,2,PE
3,2019-09-23,10:12:32.896,PAN_TC_PIU_FWL,168,48,4,0,5,3,10,0,0,0,-1,2019-09-23 10:12:32.896,6,FWL
4,2019-09-23,10:15:19.682,PAN_TC_PIU_WACL,170,0,5,0,12,32,3,169,112,0,0,2019-09-23 10:15:19.682,64,WACL
5,2019-09-23,10:15:50.701,PAN_TC_PIU_WACL,170,0,6,0,12,160,0,32,0,0,0,2019-09-23 10:15:50.701,64,WACL
6,2019-09-23,10:22:06.886,PAN_TC_PIU_WACL,170,0,7,0,12,32,3,169,112,0,0,2019-09-23 10:22:06.886,64,WACL
7,2019-09-23,10:22:26.886,PAN_TC_PIU_WACL,170,0,8,0,12,160,0,32,0,0,0,2019-09-23 10:22:26.886,64,WACL
8,2019-09-23,10:23:52.901,PAN_TC_PIU_WACL,170,0,9,0,12,32,4,225,240,0,0,2019-09-23 10:23:52.901,64,WACL
9,2019-09-23,10:24:16.901,PAN_TC_PIU_WACL,170,0,10,0,12,160,0,32,0,0,0,2019-09-23 10:24:16.901,64,WACL


In [2]:
from datetime import datetime
from astropy.time import TimeDelta, Time
from astropy import units

In [3]:
wkdir = Path(r"C:\Users\ucasbwh\Desktop\20190923_101047_WAC_L10_20C_ABSR\PROC")
RTM = pd.read_pickle(wkdir / "Unproc_HKTM.pickle")
RTM.head()

,Time,RAW,Source
0,2019-09-23\t10:11:05.656,15 07 00 1E E1 B9 00 53 00 00 4D 00 07 18 04 8...,LabView
1,2019-09-23\t10:11:06.657,15 07 00 1E E1 BA 00 53 00 00 4D 00 07 18 04 8...,LabView
2,2019-09-23\t10:11:07.656,15 07 00 1E E1 BB 00 53 00 00 4D 00 07 18 04 8...,LabView
3,2019-09-23\t10:11:08.656,15 07 00 1E E1 BC 00 53 00 00 4D 00 07 18 04 8...,LabView
4,2019-09-23\t10:11:09.656,15 07 00 1E E1 BD 00 53 00 00 4D 00 07 18 04 8...,LabView


In [4]:
Bin = RTM['RAW'].apply(lambda x: bytearray.fromhex(x))
RTM = PC_Fns.ReturnCUC_RAW(RTM, Bin)

In [5]:
RTM['DT'] = pd.to_datetime(RTM['Time'], format='%Y-%m-%d\t%H:%M:%S.%f')
RTM.head()

,Time,RAW,Source,Pkt_CUC,DT
0,2019-09-23\t10:11:05.656,15 07 00 1E E1 B9 00 53 00 00 4D 00 07 18 04 8...,LabView,132636016723,2019-09-23 10:11:05.656
1,2019-09-23\t10:11:06.657,15 07 00 1E E1 BA 00 53 00 00 4D 00 07 18 04 8...,LabView,132636082259,2019-09-23 10:11:06.657
2,2019-09-23\t10:11:07.656,15 07 00 1E E1 BB 00 53 00 00 4D 00 07 18 04 8...,LabView,132636147795,2019-09-23 10:11:07.656
3,2019-09-23\t10:11:08.656,15 07 00 1E E1 BC 00 53 00 00 4D 00 07 18 04 8...,LabView,132636213331,2019-09-23 10:11:08.656
4,2019-09-23\t10:11:09.656,15 07 00 1E E1 BD 00 53 00 00 4D 00 07 18 04 8...,LabView,132636278867,2019-09-23 10:11:09.656


In [6]:
# First extract the year and month
RTM['Year'] = RTM['DT'].dt.year
RTM['Month'] = RTM['DT'].dt.month
year = RTM['Year'][0]
month = RTM['Month'][0]

In [7]:
RTM['CUCfrac'] = RTM['Pkt_CUC'].apply(lambda x: (x & 0xFFFF)/0x10000)
test = RTM.iloc[3]
CalcTime = (Time(datetime(year, month, 1, 0, 0, 0)) - TimeDelta(1, format='jd') + TimeDelta((test['Pkt_CUC'] >> 16)*units.s) + TimeDelta(test['CUCfrac'], format='sec')).iso
print(CalcTime)

2019-09-23 10:11:08.001


In [9]:
RTM['CUCfrac'] = RTM['Pkt_CUC'].apply(lambda x: (x & 0xFFFF)/0x10000)

CalcTime = RTM.apply(lambda row: (Time(datetime(row['Year'], row['Month'], 1, 0, 0, 0)) - TimeDelta(1, format='jd') + TimeDelta((row['Pkt_CUC'] >> 16)*units.s) + TimeDelta(row['CUCfrac'], format='sec')).iso, axis=1)
print(CalcTime)

0       2019-09-23 10:11:05.001
1       2019-09-23 10:11:06.001
2       2019-09-23 10:11:07.001
3       2019-09-23 10:11:08.001
4       2019-09-23 10:11:09.001
                 ...           
1609    2019-09-23 10:37:54.001
1610    2019-09-23 10:37:55.001
1611    2019-09-23 10:37:56.001
1612    2019-09-23 10:37:57.001
1613    2019-09-23 10:37:58.001
Length: 1614, dtype: object
